In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
# --- Étape 1: Chargement, Nettoyage et Tokenization ---

# Charger les données d'entraînement et de test
train_data = pd.read_csv(train_file_path, sep='\t', header=None, names=['label', 'message'])
test_data = pd.read_csv(test_file_path, sep='\t', header=None, names=['label', 'message'])

# Convertir les labels en numérique (0 pour ham, 1 pour spam)
train_labels = train_data['label'].apply(lambda x: 1 if x == 'spam' else 0).values
test_labels = test_data['label'].apply(lambda x: 1 if x == 'spam' else 0).values
train_messages = train_data['message'].values
test_messages = test_data['message'].values

# Paramètres de Tokenization et de Séquence
vocab_size = 10000  # Taille maximale du vocabulaire
embedding_dim = 16
maxlen = 200        # Longueur maximale des séquences (padding)
trunc_type = 'post'
padding_type = 'post'
oov_tok = "<OOV>"   # Jeton pour les mots hors vocabulaire

# 1. Tokenizer: Convertit les mots en jetons numériques
# Il est entraîné UNIQUEMENT sur les données d'entraînement.
tokenizer = keras.preprocessing.text.Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_messages)

# 2. Séquençage: Convertit les messages en séquences de nombres
train_sequences = tokenizer.texts_to_sequences(train_messages)
test_sequences = tokenizer.texts_to_sequences(test_messages)

# 3. Padding: Assure que toutes les séquences ont la même longueur (maxlen)
train_padded = keras.preprocessing.sequence.pad_sequences(train_sequences, maxlen=maxlen, padding=padding_type, truncating=trunc_type)
test_padded = keras.preprocessing.sequence.pad_sequences(test_sequences, maxlen=maxlen, padding=padding_type, truncating=trunc_type)

In [ ]:
# --- Étape 2: Création et Entraînement du Modèle ---

# Architecture du modèle
model = tf.keras.Sequential([
    # Couche d'Embedding: mappe chaque jeton à un vecteur dense de 16 dimensions
    keras.layers.Embedding(vocab_size, embedding_dim, input_length=maxlen),
    # Réduit la dimensionnalité en prenant la moyenne des vecteurs (plus simple que le RNN/LSTM)
    keras.layers.GlobalAveragePooling1D(),
    # Couche dense (Fully Connected)
    keras.layers.Dense(24, activation='relu'),
    # Couche de sortie: 1 neurone pour classification binaire (0 ou 1)
    keras.layers.Dense(1, activation='sigmoid')
])

# Compilation du modèle
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Entraînement
num_epochs = 30 # Un bon nombre pour ce type de défi
history = model.fit(
    train_padded,
    train_labels,
    epochs=num_epochs,
    validation_data=(test_padded, test_labels),
    verbose=0
)

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):

    # 1. Préparer le texte brut pour le modèle:
    # Tokenizer le message, en utilisant le même tokenizer entraîné.
    sequence = tokenizer.texts_to_sequences([pred_text])
    # Padding de la séquence pour qu'elle ait la même longueur que l'entraînement.
    padded_sequence = keras.preprocessing.sequence.pad_sequences(
        sequence,
        maxlen=maxlen,
        padding=padding_type,
        truncating=trunc_type
    )

    # 2. Prédiction de la probabilité (sortie du modèle)
    # Le modèle donne une probabilité de "spam" (1)
    prediction_prob = model.predict(padded_sequence)[0][0]

    # 3. Déterminer le label (ham ou spam)
    # Si la probabilité est > 0.5, c'est spam (1); sinon, c'est ham (0)
    label = "spam" if prediction_prob > 0.5 else "ham"

    # 4. Formater la sortie comme exigé
    prediction = [prediction_prob, label]

    return prediction

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
